In [ ]:
import numpy as np
import pandas as pd
# import matplotlib.pylot as plt
# import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from skleran import metrics

In [ ]:
df=pd.read_csv('Test.csv')

In [ ]:
df.head()

In [ ]:
df.is_null